# Gallup World Poll Data Cleaning and Codebook Generation

This notebook outlines the process of cleaning the Gallup World Poll dataset and generating a comprehensive codebook. The dataset consists of numerous variables, and the objective is to streamline and clean the data for more straightforward analyses. The code is written in R 4.0.3.

## Load Required Libraries

In [6]:
# Load necessary libraries
# If any library is missing, please install it using install.packages("package_name")
myPackages <- c("dplyr", "tidyr", "glue")
lapply(myPackages, library, character.only = TRUE)
library(sjlabelled, lib.loc = "~/rlib")

[[1]]
 [1] "sjlabelled" "glue"       "tidyr"      "dplyr"      "stats"     
 [6] "graphics"   "grDevices"  "utils"      "datasets"   "methods"   
[11] "base"      

[[2]]
 [1] "sjlabelled" "glue"       "tidyr"      "dplyr"      "stats"     
 [6] "graphics"   "grDevices"  "utils"      "datasets"   "methods"   
[11] "base"      

[[3]]
 [1] "sjlabelled" "glue"       "tidyr"      "dplyr"      "stats"     
 [6] "graphics"   "grDevices"  "utils"      "datasets"   "methods"   
[11] "base"

## Loading Datasets

In [3]:
# Import the Gallup World Poll dataset
galluprds <- rio::import("~/gallup data/Gallup Raw Data/Gallup_World_Poll_021524.rds")

In [4]:
# We create a copy of the main dataset to perform extended processing without affecting the original data.
gallupExt <- galluprds
dim(gallupExt) #check its dimensions

[1] 2734564    2714

In [5]:
# Load the Gallup Questions dataset and inspect its first few rows
gallupQuestions <- rio::import("GallupQuestions240513.csv")[-3]
glimpse(gallupQuestions)

Rows: 2,706
Columns: 2
$ QTag            <chr> "COUNTRY_ISO2", "COUNTRY_ISO3", "COUNTRYNEW", "DEGURBA…
$ `Question Text` <chr> "Country ISO alpha-2 code", "Country ISO alpha-3 code"…


## For each column in the Gallup dataset, extract labels, values, and other details. 

In [7]:
Gallup.responses <- lapply(colnames(gallupExt), 
                       function(var){ 
    labels <- get_labels(gallupExt[, var])
    values <- get_values(gallupExt[, var])
        return(c(QTag = var,
                 "Short Text" = get_label(gallupExt[, var]),
                 "Response Options" = paste(labels, collapse = ", "),
                 "Response Values" = paste(values, collapse = ", ")))
                           }) %>% bind_rows

In [8]:
## Display the number of raws and the last few rows of the Gallup.responses dataframe
nrow(Gallup.responses)
tail(Gallup.responses)

[1] 2714

QTag,Short Text,Response Options,Response Values
<chr>,<chr>,<chr>,<chr>
WP9949,Voting Affects Change,"Voting is an effective way to affect positive change, Elections dont make much of a difference, (DK), (Refused)","1, 2, 3, 4"
WP9950,Know When Presidential Election Held,"Yes, No, (DK), (Refused)","1, 2, 3, 4"
WP9951,Upcoming Election Fair,"Yes, No, (DK), (Refused)","1, 2, 3, 4"
WP9983,Closer Relation With China or India,"India, China, DO NOT READ: (Both), DO NOT READ: (Neither), (DK), (Refused)","1, 2, 3, 4, 5, 6"
YEAR_CALENDAR,Calendar Year,,
YEAR_WAVE,Wave Year,,


## Identifying NA: DK, Refused, NA, No Answer, etc

We define regular expressions (regex) patterns to identify different types of non-answers like 'Do not know (DK)', 'Refused', and various forms of 'Not Available (NA)'.

Note that we may haven't gather all the NA patterns. We may have to modify this code later.

You can also check NAandBinaryResponsesInvestigation.ipynb for some insights.

In [9]:
dk.text <- c("(DK(\\b|\\W))|(((Do\\Wnot)|dont|don\\?t)\\Wknow\\W{0,1}$)")
refused.text <- c("(refuse)|(RF(\\b|\\W))")
na.text <- paste("(\\(does\\Wnot\\Wapply)",
                 "(Not\\WApplicable)",
                 "(^NA$)",
                 "(\\(N\\W{0,1}A\\))",
                 "(No\\Wanswer)", 
                 "(No\\Wresponse)",
                 "(Not\\WAvailable\\W{0,1}$)", sep = "|") 
texttoNA <- paste(dk.text, refused.text, na.text, sep = "|")

In [10]:
# Create a dataframe with the details of questions with non-answer responses
Gallup.NA <- colnames(gallupExt) %>% 
    # Apply a function to each column name.
    lapply(function(var){ 
        # Get the labels(response options) and the values(response values)
        # of the current column
        labels <- get_labels(gallupExt[, var])
        values <- get_values(gallupExt[, var])
        
        # Identify the value corresponding to the non-answers.
        dk.value <- values[grepl(dk.text, labels, ignore.case = T)]
        refused.value <- values[grepl(refused.text, labels, ignore.case = T)]
        na.value <- values[grepl(na.text, labels, ignore.case = T)]
        
        # only return the details when there are non-answers
        if (sum(grepl(texttoNA, labels, ignore.case = T)) > 0 &
            ! var %in% "COUNTRY_ISO2"){
            return(c(QTag = var,
                     "Short Text" = get_label(gallupExt[, var]),
                     "(dk).value" = dk.value,
                     "(refused).value" = refused.value,
                     "(na).value" = na.value,
                     "Response Options" = paste(labels, collapse = ", "),
                     "Response Values" = paste(values, collapse = ", ")))
                               }}) %>% 
    # Combine the results of the 'lapply' function into a single data frame.
    bind_rows %>%
    # rearrange the columns into a more readable format
    select(QTag, `Short Text`, `(dk).value`, `(refused).value`, 
           `(na).value`, `Response Options`, `Response Values`)

In [11]:
#2023 June 20th, WP12 has the option 97: Not Available but not shown in the label of original sav file,
#so we add it here
cat("attributes of WP12\n")
attributes(gallupExt$WP12)

cat("\nvalues existed in the data but without label:")
unique(na.omit(gallupExt$WP12)) %>% .[!. %in% get_values(gallupExt$WP12)]

cat("\nAdd the info into Gallup.NA df")
Gallup.NA[Gallup.NA$QTag %in% "WP12", "(na).value"] <- "97"
Gallup.NA[Gallup.NA$QTag %in% "WP12",]

attributes of WP12


$label
[1] "Residents 15+ in Household"

$format.spss
[1] "F8.0"

$labels
                                                                                
        1         2         3         4         5         6         7         8 
                                                                                
        9        10        11        12        13        14        15        16 
                                                                                
       17        18        19        20        21        22        23        24 
                                                                                
       25        26        27        28        29        30        31        32 
                                                                                
       33        34        35        36        37        38        39        40 
                                                                                
       41        42        43        44        45        46        47        48 
                                                                                
       49        50        51        52        53        54        55        56 
                                                                                
       57        58        59        60        61        62        63        64 
                                                                                
       65        66        67        68        69        70        71        72 
                                                                                
       73        74        75        76        77        78        79        80 
                                                                                
       81        82        83        84        85        86        87        88 
                                                                            96+ 
       89        90        91        92        93        94        95        96 
     (DK) (Refused) 
       98        99


values existed in the data but without label:

[1]  0 97


Add the info into Gallup.NA df

QTag,Short Text,(dk).value,(refused).value,(na).value,Response Options,Response Values
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
WP12,Residents 15+ in Household,98,99,97,", , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , 96+, (DK), (Refused)","1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 98, 99"


In [12]:
#2023 Nov 10th, I realized some REGION_XXX did not list out all the NA related label
gallupReg <- gallupExt %>% select(starts_with("REGION_"))

# Loop through each column
# the values printed with "*" are missing their labels
for (var in names(gallupReg)) {
    # Check if the column has 'labels' attribute
    if (!is.null(attr(gallupReg[, var], "labels"))) {
        # Create a mapping from labels
        labels <- get_labels(gallupReg[, var])
        values <- get_values(gallupReg[, var])

        # Find those values missing the label
        values_unique <- unique(na.omit(gallupReg[, var])) %>% .[order(.)]
        if(sum(!values_unique %in% values) > 0){
            values_print <- paste(values, labels)
            values_print[!values %in% values_unique] <- paste0("\n\t", values_print[!values %in% values_unique], " *no occurrence in the data")
            values_print <- c(values_print, paste0("\n\t", "(", values_unique[!values_unique %in% values], ") *missing label"))
            cat(paste0(var, ":\n\t", paste0(values_print, collapse = ", "), "\n --------------- \n"))
        }
    }
}

REGION_MLT:
	1 Southern Harbour District, 2 Northern Harbour District, 3 South Eastern District, 4 Western District, 5 North District, 6 Gozo and Comino, 
	(99) *missing label
 --------------- 
REGION_NCY:
	1 Famagusta, 2 Iskele, 3 Kyrenia, 4 Morfou, 5 Nicosia, 
	8 (DK) *no occurrence in the data, 
	9 (Refused) *no occurrence in the data, 
	(99) *missing label
 --------------- 
REGION_OMN:
	1 Ad Dakhiliyah, 2 Ad Dhahirah, 3 Al Batinah, 
	4 Al Buraymi *no occurrence in the data, 5 Al Wusta, 6 Ash Sharqiyah, 7 Dhofar, 8 Musandam, 9 Muscat, 
	(98) *missing label, 
	(99) *missing label
 --------------- 


In [13]:
#so I'm including them here
varsWithIssue <- c("REGION_MLT", "REGION_NCY", "REGION_OMN")
varsWithIssue_notIn <- varsWithIssue[!varsWithIssue %in% Gallup.NA$QTag] # the variables that not yet in the Gallup.NA
Gallup.NA <- Gallup.NA %>% bind_rows(Gallup.responses[Gallup.responses$QTag %in% varsWithIssue_notIn,])

Gallup.NA[Gallup.NA$QTag %in% "REGION_MLT", "(refused).value"] <- "99"
Gallup.NA[Gallup.NA$QTag %in% "REGION_NCY", "(refused).value"] <- "99"
Gallup.NA[Gallup.NA$QTag %in% "REGION_OMN", "(dk).value"] <- "98"
Gallup.NA[Gallup.NA$QTag %in% "REGION_OMN", "(refused).value"] <- "99"

#review the result
Gallup.NA[Gallup.NA$QTag %in% varsWithIssue,]

QTag,Short Text,(dk).value,(refused).value,(na).value,Response Options,Response Values
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
REGION_NCY,Region North Cyprus,8,99,NA,"Famagusta, Iskele, Kyrenia, Morfou, Nicosia, (DK), (Refused)","1, 2, 3, 4, 5, 8, 9"
REGION_MLT,Region Malta,NA,99,NA,"Southern Harbour District, Northern Harbour District, South Eastern District, Western District, North District, Gozo and Comino","1, 2, 3, 4, 5, 6"
REGION_OMN,Region Oman,98,99,NA,"Ad Dakhiliyah, Ad Dhahirah, Al Batinah, Al Buraymi, Al Wusta, Ash Sharqiyah, Dhofar, Musandam, Muscat","1, 2, 3, 4, 5, 6, 7, 8, 9"


## Identify questions with a non-zero "None" response

We investigate questions that have the option 'None' with non-zero values. While these are not necessarily considered as NA, it's essential to be cautious when interpreting them.

In [14]:
none.text <-c("(\\W|\\b)none(\\W|\\b)")
# Create a dataframe with the details of questions with non-zero "None" responses
Gallup.none <- lapply(colnames(gallupExt), 
    # Apply a function to each column name.
                       function(var){
    # Get the labels(response options) and the values(response values)
    # of the current column
    values <- get_values(gallupExt[, var])                       
    labels <- get_labels(gallupExt[, var])
                           
    # Identify all the values corresponding to "none".                       
    none.values <- values[grepl(none.text, labels, ignore.case = T)]
    # Identify the values corresponding to non-zero "none" and the exact text                     
    none.value <- none.values[none.values != 0]  
    none.label <- labels[values %in% none.value]
    
    # only return the details when there are non-zero "none"                      
    if (length(none.value) > 0){
        return(c(QTag = var,
                 "Short Text" = get_label(gallupExt[, var]),
                 "non-zero.none.value" = paste(none.value, collapse = ", "),
                 "Response Options" = paste(labels, collapse = ", "),
                 "Response Values" = paste(get_values(gallupExt[, var]), collapse = ", ")))
                           }}) %>% bind_rows 
nrow(Gallup.none)

[1] 78

## Identifying Questions with Binary Responses (1 and 2)

We aim to identify questions that have only two valid responses, sometimes represented by 1 for affirmation and 2 for negation. This excludes non-answer like DK/Refused. By pinpointing these questions, we can modify the response options from {1,2} to {1,0} later. This adjustment facilitates clearer interpretation of the data.

In [15]:
# Create a dataframe with the details of questions with binary responses

Gallup.Binary <- lapply(colnames(gallupExt), 
    # Apply a function to each column name.
                       function(var){
    # Get the labels(response options) and the values(response values)
    # of the current column
    labels <- get_labels(gallupExt[, var])
    values <- get_values(gallupExt[, var])
    # Identify all values of non-NA responses                       
    na.values <- Gallup.NA[Gallup.NA$QTag %in% var, c("(dk).value", "(refused).value", "(na).value")] %>% unlist
    nonna.values <- as.numeric(values[!values %in% na.values]) %>% sort
    
    # return the details if 1 and 2 are the only non-NA responses                        
    if (identical(nonna.values, c(1, 2)))
        return(c(QTag = var,
                 "Short Text" = get_label(gallupExt[, var]),
                 "Response Options" = paste(labels, collapse = ", "),
                 "Response Values" = paste(values, collapse = ", ")))
}) %>% bind_rows %>% merge(gallupQuestions, by = "QTag")

nrow(Gallup.Binary)

[1] 1386

## Generating the Codebook

The codebook provides a comprehensive reference for all the questions in the dataset, including their response types and specific labels.

In [16]:
column.order <- c("Binary Responses = {1,2}", "QTag", "Short Text", 
                  "Question Text", 
                  "(dk).value", "(refused).value", 
                  "(na).value", "non-zero.none.value",
                  "Response Options", "Response Values")

Gallup.CodeBook <- Gallup.responses %>%
    merge(gallupQuestions, by = "QTag", all = T) %>% 
    merge(Gallup.NA %>% select(QTag, `(dk).value`, `(refused).value`, `(na).value`), by = "QTag", all = T) %>%
    merge(Gallup.none %>% select(QTag, `non-zero.none.value`),  by = "QTag", all = T) %>%
    mutate(`Binary Responses = {1,2}` = (QTag %in% Gallup.Binary$QTag))

Gallup.CodeBook <- Gallup.CodeBook[, column.order]

nrow(Gallup.CodeBook)

[1] 2717

In [17]:
Gallup.CodeBook %>% 
    group_by(`Response Values`, `Binary Responses = {1,2}`) %>% 
    {merge(slice(., 1), summarize(., `n of QTag with the corresponding Response Values` = n()))} %>% 
    arrange(nchar(`Response Values`)) %>% 
    filter(`n of QTag with the corresponding Response Values` >= 10)

`summarise()` has grouped output by 'Response Values'. You can override using the `.groups` argument.



"Binary Responses = {1,2}",Response Values,QTag,Short Text,Question Text,(dk).value,(refused).value,(na).value,non-zero.none.value,Response Options,n of QTag with the corresponding Response Values
<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
FALSE,,COUNTRY_ISO2,Country ISO alpha-2 code,Country ISO alpha-2 code,NA,NA,NA,NA,"US, EG, LB, SA, JO, TR, PK, ID, BD, GB, FR, DE, NL, BE, ES, IT, PL, HU, CZ, RO, SE, GR, DK, IR, HK, SG, JP, CN, IN, VE, BR, MX, NG, KE, TZ, IL, PS, GH, UG, BJ, MG, MW, ZA, CA, AU, PH, LK, VN, TH, KH, LA, NZ, BW, ML, MZ, NE, RW, SN, ZM, KR, TW, BY, GE, KZ, KG, MD, RU, UA, BF, CM, SL, ZW, CR, AL, AR, AM, AT, AZ, BO, BA, BG, TD, CL, CO, HR, CU, CY, DO, EC, SV, EE, FI, GT, HT, HN, IE, JM, KW, LV, LT, MK, MY, ME, NP, NI, NO, PA, PY, PE, PT, PR, RS, SK, SI, CH, TJ, TG, TT, AE, UY, UZ, YE, XK, MR, BZ, CF, GY, LR, MN, NA, SY, AF, BI, CG, DJ, IS, IQ, LU, MT, QA, CD, CI, TM, BH, KM, SD, XS, TN, MA, DZ, AO, GA, GN, LS, MU, SZ, OM, MM, ET, LY, SR, XC, BT, XN, SO, SS, GM, MV",46
TRUE,"1, 2",REGION2_NZL,Region 2 New Zealand,Region 2 New Zealand,NA,NA,NA,NA,"North Island, South Island",24
FALSE,"1, 2, 3",INCOME_7,Reported Versus Imputed Values,Reported versus imputed values,NA,NA,NA,NA,"Exact reported value, Imputed from categorical response, Imputed without categorical response",27
FALSE,"1, 2, 3, 4",REGION2_ARM,Region 2 Armenia,Region 2 Armenia,NA,NA,NA,NA,"Capital, Central Armenia, North Armenia, South Armenia",25
TRUE,"1, 2, 3, 4",M1,Not Enough Money: Healthcare,Have there been times in the past 12 months when you did not have enough money to pay for healthcare and/or medicines that you or your family needed?,3,4,NA,NA,"Yes, No, (DK), (Refused)",1250
TRUE,"1, 2, 8, 9",WP10120,Consider Self IDP,Do you consider yourself to be an IDP (internally displaced person)? (asked only of those who have moved),8,9,NA,NA,"Yes, No, (DK), (Refused)",56
FALSE,"1, 2, 3, 4, 5",INCOME_5,Per Capita Income Quintiles,Per Capita income quintiles,NA,NA,NA,NA,"Poorest 20%, Second 20%, Middle 20%, Fourth 20%, Richest 20%",313
FALSE,"1, 2, 3, 8, 9",REGION_HKG,Region Hong Kong,Region Hong Kong,8,9,NA,NA,"Hong Kong Island, Kowloon, New Territories, (DK), (Refused)",36
TRUE,"1, 2, 3, 4, 5",WP10633,Receiving Pension,Are you receiving a pension or some form of financial remuneration for work you did while you were employed? (asked only of those who are age 61 or older),3,4,5,NA,"Yes, No, (DK), (Refused), (Not Applicable)",23


In [19]:
write.csv(Gallup.CodeBook, "GallupWorldPollCodebook_GWP021524.csv", row.names = F)

## Data Cleaning Using the Codebook

Utilizing the codebook, this section cleans the Gallup dataset by processing specific response types and adjusting binary responses.

- Convert certain responses to NA
- Convert binary questions from {1,2} to {1,0}
- Update attributes of each variable to include the question text
- Save the cleaned dataset as an RDS file

In [20]:
#Create a function for changing the "DK" / "Refused" response to NA.
na_codes <- function(x, ...) {
    for(y in list(...))
        x[x == y] <- NA
    x
}

In [21]:
gallupCleaned <- gallupExt
gallupRef <- Gallup.CodeBook

for(x in colnames(gallupCleaned)){
    #converting responses to NA
    gallupCleaned[[x]] <- na_codes(gallupCleaned[[x]], 
                                 gallupRef[gallupRef$QTag %in% x, "(dk).value"],
                                 gallupRef[gallupRef$QTag %in% x, "(refused).value"],
                                 gallupRef[gallupRef$QTag %in% x, "(na).value"])
    
    #coverting responses of binary questions from {1,2} to {1,0}
    if(gallupRef[gallupRef$QTag %in% x, "Binary Responses = {1,2}"]){
        gallupCleaned[[x]] <- 2 - gallupCleaned[[x]]
    }
}

#editing the attributes of each variables to contain the text of the questions
for(x in colnames(gallupCleaned)){
   if(! class(gallupCleaned[[x]]) %in% "Date"){
        attributes(gallupCleaned[[x]]) <- NULL
    }
    attr(gallupCleaned[[x]], "Short Text") <- gallupRef[gallupRef$QTag %in% x, "Short Text"]
    attr(gallupCleaned[[x]], "Question Text") <- gallupRef[gallupRef$QTag %in% x, "Question Text"]
}

In [22]:
codebook <- lapply(colnames(gallupCleaned), function(x){
    uniqueValues <- if (!all(is.na(gallupRef[gallupRef$QTag %in% x, c("(dk).value", "(refused).value", "(na).value")])) |
                        (gallupRef[gallupRef$QTag %in% x, "Binary Responses = {1,2}"]))
        paste(sort(unique(gallupCleaned[[x]])), collapse = ", ")
                    else ""
    return(data.frame(QTag = x, ValuesAfterCleaning = uniqueValues))
}) %>% bind_rows %>% 
    merge(gallupRef, by = "QTag", all.x = T) %>% 
    relocate(ValuesAfterCleaning, .after = last_col())

colnames(codebook)[colnames(codebook) %in% c("Response Options", "Response Values")] <- c("OptionsOriginal", "ValuesOriginal")

## Saving the Cleaned Data and the Codebook with response values after cleaning

After all the processing and cleaning steps, the final cleaned dataset is saved as an RDS file for future use and analyses.

For the codebook, if it is empty in "ValuesAfterCleaning", that means nothing have changed for that variable.

In [23]:
saveRDS(gallupCleaned, "GWP_021524_FullyCleaned.rds")

In [24]:
write.csv(codebook, "Codebook_GWP_021524_FullyCleaned.csv", row.names = F)